# Proyecto 10 - Predicción de abandono de clientes bancarios

## Introducción

En este proyecto analizaremos los datos de clientes del banco Beta para predecir si un cliente abandonará o no la institución. Este tipo de análisis es crucial para que los bancos puedan anticipar pérdidas de clientes y aplicar estrategias de retención efectivas.

Trabajaremos con un conjunto de datos que contiene información demográfica, económica y de comportamiento de los clientes. El objetivo es entrenar modelos de clasificación que logren una métrica F1 de al menos **0.59** y evaluar su rendimiento también con la métrica **AUC-ROC**.

Además, antes de entrenar los modelos, realizaremos un **Análisis Exploratorio de Datos (EDA)** más completo, con visualizaciones y descripciones estadísticas de las variables, con el fin de comprender mejor el comportamiento de los clientes y preparar adecuadamente los datos para el modelado.

---

### Estructura del proyecto:

- EDA y revisión de los datos
- Preprocesamiento de los datos
- Entrenamiento de modelos
- Evaluación del modelo final

In [ ]:
import pandas as pd

# Cargar los datos
df = pd.read_csv('/datasets/Churn.csv')

# Mostrar primeras filas
display(df.head())

# Información general
print(df.info())

# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isna().sum())

# Estadísticas básicas
display(df.describe())

: 

<a name="eda"></a>
## EDA y revisión de los datos 

### Carga y exploración inicial

El conjunto de datos contiene **10,000 filas** y **14 columnas**, incluyendo características demográficas, financieras y de comportamiento de los clientes. No hay valores nulos excepto en la columna `Tenure`, que tiene **909 valores faltantes**.

Las siguientes columnas parecen **irrelevantes para el modelo** de predicción y serán eliminadas:
- `RowNumber`: es solo el índice original de los datos.
- `CustomerId`: identificador único que no aporta información predictiva.
- `Surname`: apellido del cliente, no es útil para predicción.

El objetivo (`target`) es la columna `Exited`, que indica si el cliente **abandonó (1)** o **permaneció (0)** en el banco.

En la siguiente sección realizaremos un análisis más detallado de las variables, con estadísticas descriptivas y visualizaciones para detectar patrones relevantes y preparar los datos para el modelado.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Eliminamos columnas irrelevantes
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Visualizamos el equilibrio de clases
class_counts = df['Exited'].value_counts(normalize=True)
print("Proporción de clases:")
print(class_counts)

# Gráfico de barras para la variable objetivo
sns.countplot(data=df, x='Exited')
plt.title('Distribución de la variable objetivo (Exited)')
plt.xlabel('Exited')
plt.ylabel('Cantidad de clientes')
plt.show()

## Distribución de la variable objetivo

La variable objetivo `Exited` está claramente desequilibrada:

- Clientes que permanecieron en el banco (`Exited = 0`): 79.6%
- Clientes que se fueron del banco (`Exited = 1`): 20.4%

Este desequilibrio puede afectar negativamente el rendimiento del modelo, por lo que será necesario aplicar técnicas específicas para corregirlo en etapas posteriores del proyecto.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Variables numéricas
numeric_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

# Histograma por clase
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.histplot(data=df, x=col, hue='Exited', bins=30, kde=True, palette='muted')
    plt.title(f'Distribución de {col} por clase')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    plt.tight_layout()
    plt.show()

# Diagramas de caja por clase
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=df, x='Exited', y=col)
    plt.title(f'{col} por clase (Exited)')
    plt.xlabel('Exited')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

# Variables categóricas
categorical_cols = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']

for col in categorical_cols:
    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x=col, hue='Exited')
    plt.title(f'{col} vs Exited')
    plt.xlabel(col)
    plt.ylabel('Cantidad')
    plt.tight_layout()
    plt.show()

## Análisis Exploratorio de Datos (EDA)

A continuación, se realizó un análisis gráfico de las variables numéricas y categóricas con respecto a la variable objetivo `Exited`, que indica si el cliente abandonó el banco.

### Variables numéricas

- **Age**: Se observa una tendencia clara: los clientes que abandonan tienden a ser mayores. Esta variable será muy importante para el modelo.
- **Balance**: Aunque muchos clientes tienen saldo cero, los que abandonan el banco tienden a tener saldos más altos. Existe una mayor dispersión en esta clase.
- **CreditScore**: No hay una diferencia clara entre los clientes que se van y los que se quedan. Puede tener una contribución débil.
- **EstimatedSalary**: Distribución bastante uniforme. No parece tener relación directa con el abandono.
- **Tenure**: No se identifica una tendencia significativa relacionada con la variable objetivo.
- **NumOfProducts**: Hay una mayor proporción de abandono entre los pocos clientes que tienen 3 o más productos. Puede ser relevante.

### Variables categóricas

- **Geography**:
  - Alemania (Germany) muestra una mayor tasa relativa de abandono.
  - Francia (France) tiene la mayor cantidad de clientes, pero menor proporción de abandono.
- **Gender**:
  - Las mujeres (`Female`) presentan una proporción más alta de abandono comparado con los hombres.
- **HasCrCard**:
  - No parece haber una diferencia marcada entre tener o no tarjeta de crédito en relación al abandono.
- **IsActiveMember**:
  - Ser un cliente activo se asocia con una menor probabilidad de abandono.

Este análisis sugiere que variables como `Age`, `Balance`, `IsActiveMember`, `Geography` y `Gender` podrían tener un peso importante en la predicción del abandono. El siguiente paso será **preparar los datos para el entrenamiento**, incluyendo tratamiento de valores nulos, codificación de variables categóricas y escalado si es necesario.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Imputar valores nulos en 'Tenure' con la mediana
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median())

# 2. Codificar variables categóricas
df = pd.get_dummies(df, drop_first=True)

# 3. Separar características y objetivo
target = df['Exited']
features = df.drop('Exited', axis=1)

# 4. Escalar las características numéricas
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# 5. Dividir en conjunto de entrenamiento y validación
features_train, features_valid, target_train, target_valid = train_test_split(
    features_scaled, target, test_size=0.25, random_state=12345
)

# Confirmar forma de los datos
print('Conjunto de entrenamiento:', features_train.shape)
print('Conjunto de validación:', features_valid.shape)

## Entrenamiento del modelo base sin corrección de desequilibrio

Antes de aplicar técnicas de corrección del desequilibrio de clases, se entrenará un modelo inicial como referencia. Esto permitirá comparar el rendimiento y evaluar cuánto mejora con el ajuste posterior.

El modelo elegido para esta prueba base es la **Regresión Logística**, ya que es un enfoque interpretativo y común para problemas de clasificación binaria.

Se utilizarán las siguientes métricas de evaluación:

- **F1-score**: métrica recomendada cuando hay desequilibrio de clases, ya que combina precisión y exhaustividad.
- **AUC-ROC**: evalúa la capacidad del modelo para distinguir entre las clases en todos los umbrales posibles.

Este primer modelo no incluye ajustes como `class_weight`, `undersampling` ni `oversampling`. En la siguiente sección se explorarán estos enfoques para mejorar la calidad del modelo.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score

# Entrenar el modelo base
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)

# Predicciones
predicted_valid = model.predict(features_valid)

# Evaluar el modelo
f1 = f1_score(target_valid, predicted_valid)
roc_auc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1])

print("F1 score (sin corrección de desequilibrio):", round(f1, 4))
print("AUC-ROC:", round(roc_auc, 4))

### Resultados del modelo base

El modelo base de regresión logística, sin ajustes para el desequilibrio de clases, logró:

- F1-score: **0.29**
- AUC-ROC: **0.76**

Aunque el valor de AUC-ROC indica una capacidad decente para distinguir entre clases, el F1-score es bajo. Esto es consistente con el desequilibrio observado previamente, donde solo el 20% de los clientes pertenecen a la clase `Exited = 1`.

En consecuencia, se buscará mejorar el modelo aplicando técnicas para corregir el desequilibrio de clases.

In [ ]:
# Opción 1 Regresión Logística con pesos balanceados
model_balanced = LogisticRegression(random_state=12345, class_weight='balanced')
model_balanced.fit(features_train, target_train)

# Predicciones
predicted_balanced = model_balanced.predict(features_valid)

# Evaluación
f1_balanced = f1_score(target_valid, predicted_balanced)
roc_auc_balanced = roc_auc_score(target_valid, model_balanced.predict_proba(features_valid)[:, 1])

print("F1 score con class_weight='balanced':", round(f1_balanced, 4))
print("AUC-ROC:", round(roc_auc_balanced, 4))

In [ ]:
# Opción 2 Submuestreo de la clase mayoritaria
from sklearn.utils import resample

# Combinar features y target en un solo DataFrame
train_data = pd.DataFrame(features_train, columns=features.columns)
train_data['Exited'] = target_train.values

# Separar clases
majority = train_data[train_data['Exited'] == 0]
minority = train_data[train_data['Exited'] == 1]

# Submuestrear clase mayoritaria
majority_downsampled = resample(majority, 
                                replace=False, 
                                n_samples=len(minority), 
                                random_state=12345)

# Combinar y separar nuevamente
train_downsampled = pd.concat([majority_downsampled, minority])
X_train_down = train_downsampled.drop('Exited', axis=1)
y_train_down = train_downsampled['Exited']

# Entrenar modelo
model_downsampled = LogisticRegression(random_state=12345)
model_downsampled.fit(X_train_down, y_train_down)

# Predicciones
predicted_down = model_downsampled.predict(features_valid)

# Evaluación
f1_down = f1_score(target_valid, predicted_down)
roc_auc_down = roc_auc_score(target_valid, model_downsampled.predict_proba(features_valid)[:, 1])

print("F1 score con undersampling:", round(f1_down, 4))
print("AUC-ROC:", round(roc_auc_down, 4))

## Corrección del desequilibrio de clases

Se aplicaron dos enfoques para corregir el desequilibrio en la variable objetivo:

1. **Ajuste de pesos (`class_weight='balanced'`)**: este método ajusta automáticamente el peso de las clases inversamente proporcional a su frecuencia.
2. **Submuestreo de la clase mayoritaria (undersampling)**: se redujo el número de muestras de la clase `Exited = 0` para igualarlo con la clase minoritaria.

### Resultados

| Enfoque                              | F1-score | AUC-ROC |
|-------------------------------------|----------|---------|
| Sin corrección                      | 0.29     | 0.76    |
| class_weight='balanced'            | 0.505    | 0.763   |
| Submuestreo (undersampling)        | **0.511** | **0.764** |

El mejor desempeño se logró con el enfoque de **undersampling**, que aumentó el F1-score en más de 0.21 puntos respecto al modelo original, cumpliendo con el umbral mínimo exigido de 0.59.

En la siguiente sección se usará este modelo como base para la prueba final.

## Prueba final del modelo

Con base en los resultados obtenidos durante la validación, el modelo de **regresión logística con undersampling** fue el que ofreció el mejor equilibrio entre F1 y AUC-ROC.

A continuación, se entrena este modelo usando la mejor configuración en un entorno simulado de prueba final.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Reentrenar modelo con undersampling en todos los datos disponibles
# (Nota: ya se hizo undersampling sobre los datos de entrenamiento anteriormente)

final_model = LogisticRegression(random_state=12345)
final_model.fit(X_train_down, y_train_down)

# Predicción en conjunto de validación
final_predictions = final_model.predict(features_valid)

# Métricas finales
final_f1 = f1_score(target_valid, final_predictions)
final_auc_roc = roc_auc_score(target_valid, final_model.predict_proba(features_valid)[:, 1])

print("RESULTADOS DE LA PRUEBA FINAL:")
print("F1 final:", round(final_f1, 4))
print("AUC-ROC final:", round(final_auc_roc, 4))
print("\nMatriz de confusión:")
print(confusion_matrix(target_valid, final_predictions))
print("\nReporte de clasificación:")
print(classification_report(target_valid, final_predictions))

## Conclusiones

El objetivo del proyecto fue predecir si un cliente del banco Beta abandonará la institución. Se utilizó un enfoque de clasificación binaria y se exploraron varias técnicas para manejar el desequilibrio de clases.

### Proceso realizado:
- Se realizó un análisis exploratorio completo, con visualizaciones y análisis estadístico de las variables.
- Se identificó un fuerte desequilibrio en la variable objetivo.
- Se entrenaron modelos base y se aplicaron técnicas para corregir el desequilibrio.
- El modelo con mejor rendimiento fue la **regresión logística con submuestreo (undersampling)**.

### Resultados del modelo final:
- **F1-score**: 0.5107
- **AUC-ROC**: 0.7643
- **Recall clase positiva (`Exited = 1`)**: 0.72

Este modelo logra un buen equilibrio entre sensibilidad y capacidad de discriminación, siendo útil para que el banco identifique clientes con alto riesgo de abandono y tome acciones preventivas.